### 클롤링 함수 만들고, 데이터 수집하기

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import time, datetime
import unicodedata

In [ ]:
driver = webdriver.Chrome("Desktop/chromedriver")
url = 'https://www.instagram.com/'
driver.get(url)

In [ ]:
# 첫 페이지 선택하기
driver.find_element_by_css_selector("div._aagu").click()

In [ ]:
# 크롤링 함수
def searching_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select('div._a9zs > span._aacl')[0].text
        content = unicodedata.normalize('NFC', content)
    
    except:
        content = ''
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    time = soup.select('time._aaqe')[0]['datetime'][:10]
    
    try:
        place = soup.select('div._aaqm')[0].text
        
    except:
        place = ' '
    
    data = [content, tags, time, place]
    
    return data

In [ ]:
# 다음 페이지 함수
def move_next(driver):
    right = driver.find_element_by_css_selector('div._aaqg')
    right.click()
    time.sleep(5)

In [ ]:
# 독립기념관 크롤링
results_8 = []
target = 1000

for i in range(target):
    try:
        data = searching_content(driver)
        results_815.append(data)
        move_next(driver)
        
    except:
        move_next(driver)
        time.sleep(5)

In [ ]:
results_815

In [ ]:
results_815_df = pd.DataFrame(results_815)
results_815_df.columns = ['내용', '태그', '날짜', '장소']

In [ ]:
results_815_df

In [ ]:
results_815_df.to_excel("Desktop/독립기념관SNS.xlsx", index = False)

In [ ]:
# 군인 휴가 크롤링
results_vacation = []
target = 1000

for i in range(target):
    try:
        data = searching_content(driver)
        results_vacation.append(data)
        move_next(driver)
        
    except:
        move_next(driver)
        time.sleep(3)

In [ ]:
results_vacations_df = pd.DataFrame(results_vacation)

In [ ]:
results_vacations_df.columns = ['내용', '해시태그', '날짜', '위치']

In [ ]:
results_vacations_df.to_excel("Desktop/army_vacation.xlsx", index = False)

In [ ]:
SNS_815 = pd.read_excel("Desktop/독립기념관SNS.xlsx")

### 독립기념관 크롤링 데이터 전처리

In [ ]:
SNS_815['태그']

In [ ]:
tags_total = []

for tags in SNS_815['태그']:
    tags_list = tags[2:-2].split(",")
    for tag in tags_list:
        tags_total.append(tag)

In [ ]:
# 빈도주 집계(Counter)
from collections import Counter
tags_counts = Counter(tags_total)

# 가장 많이 사용된 해시태그 확인
tags_counts.most_common(100)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import sys

if sys.platform == 'darwin':
    font_name = "AppleGothic"
    
rc('font', family = font_name)

In [ ]:
tag_counts_df = pd.DataFrame(tags_counts.most_common(30))
tag_counts_df.columns = ['tags', 'counts']

In [ ]:
plt.figure(figsize = (10, 8))
sns.barplot(x = 'counts', y = 'tags', data = tag_counts_df)

### 군인 크롤링 데이터 전처리

In [ ]:
SNS_army = pd.read_excel("Desktop/army_vacation.xlsx")

In [ ]:
army_tags = []

for tag in SNS_army['해시태그']:
    tag_list = tag[2:-2].split(",")
    for tag in tag_list:
        army_tags.append(tag)

In [ ]:
army_tags_counts = Counter(army_tags)

In [ ]:
army_tags_counts.most_common()

In [ ]:
army_tag_counts_df = pd.DataFrame(army_tags_counts.most_common(30))
army_tag_counts_df.columns = ['tags', 'counts']

In [ ]:
plt.figure(figsize = (10, 8))
sns.barplot(x = 'counts', y = 'tags', data = army_tag_counts_df)

### 지역 투어 상품 데이터

In [ ]:
tour_data_df = pd.DataFrame([])

folder = 'Desktop/지역별 투어:교통상품 데이터/'
f_list = ['YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202209.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202208.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202207.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202206.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202205.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202203.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202202.csv',
         'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_202201.csv']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_csv(fpath)
    tour_data_df = tour_data_df.append(temp)
    
tour_data_df.columns = ['지역', '투어상품코드', '상품명', '상품주소', '날짜']

In [ ]:
tour_data_df = tour_data_df.dropna()

In [ ]:
tour_cheonan_df = tour_data_df[tour_data_df['지역'].str.contains('천안')]

In [ ]:
tour_cheonan_df.to_excel("Desktop/야놀자 천안 인기 여행지.xlsx", index = False)

### 여행지 키워드 데이터

In [ ]:
tour_keyword_data_df = pd.DataFrame([])

f_list = ['CI_YEAR_ACCTO_TRRSRT_KWRD_LIST_2021.csv', 'CI_ALL_PD_TRRSRT_KWRD_INFO_2020.csv']

folder = 'Desktop/여행지 키워드 데이터/'

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_csv(fpath)
    tour_keyword_data_df = tour_keyword_data_df.append(temp)

In [ ]:
tour_keyword_data_df

In [ ]:
tour_keyword_all_df = tour_keyword_data_df[tour_keyword_data_df['AREA_NM'] == '전국']

In [ ]:
tour_keyword_all_df = tour_keyword_all_df.fillna('2020')
tour_keyword_all_df = tour_keyword_all_df[['AREA_NM', 'KWRD_RANK_CO', 'KWRD_NM', 'OCCRRNC_YEAR']]
tour_keyword_all_df.columns = ['지역', '순위', '여행키워드', '년도']

In [ ]:
tour_keyword_all_df.to_excel("Desktop/전국 여행지 키워드.xlsx", index = False)

In [ ]:
tour_keyword_cheonan_df = tour_keyword_data_df[tour_keyword_data_df['AREA_NM'] == '충청남도']

In [ ]:
tour_keyword_cheonan_df = tour_keyword_cheonan_df.fillna('2020.0')

In [ ]:
tour_keyword_cheonan_df = tour_keyword_cheonan_df[['AREA_NM', 'KWRD_RANK_CO', 'KWRD_NM', 'OCCRRNC_YEAR']]
tour_keyword_cheonan_df.columns = ['지역', '순위', '여행키워드', '년도']
tour_keyword_cheonan_df

In [ ]:
tour_keyword_cheonan_df.to_excel("Desktop/충청남도 여행지 키워드.xlsx", index = False)

### 여가시간 활용 목적 데이터

In [ ]:
time_data_df = pd.DataFrame([])

f_list = ['CI_LSR_TIME_USE_PURPS_INFO_20220720.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220627.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220629.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220704.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220718.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220725.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220727.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220803.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220810.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220817.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220824.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220831.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220907.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220914.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220921.csv',
         'CI_LSR_TIME_USE_PURPS_INFO_20220928 (1).csv']

folder = 'Desktop/여가시간 활용목적/'

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_csv(fpath)
    time_data_df = time_data_df.append(temp)

In [ ]:
time_data_df.columns = ['응답자ID', '조사시작일자', '성별', '연령대', '거주지', '가구소득정도', '여가시간1순위', '여가시간2순위']

In [ ]:
time_20M_df = time_data_df[(time_data_df['성별'] == 'M') & (time_data_df['연령대'] == '20대')]

In [ ]:
time_20M_df['여가시간1순위'].value_counts()

In [ ]:
time_20M_df['여가시간2순위'].value_counts()

In [ ]:
time_20M_df.to_excel("Desktop/20대 남성 여가시간.xlsx", index =False)

In [ ]:
pd.read_excel("Desktop/20대 남성 여가시간.xlsx")

### 천안의 흥 먹거리 크롤링 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
    url = "https://www.cheonan.go.kr/prog/hotFood/tour/sub03_08/list.do?area=&pageIndex={}".format(i)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
soup.select

In [ ]:
rest_list

In [ ]:
rest_list_data = []

for i in range(1, 6):
    url = "https://www.cheonan.go.kr/prog/hotFood/tour/sub03_08/list.do?area=&pageIndex={}".format(i)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    rest_list = soup.select('span.con')

    for i in rest_list:
        menu = i.select('li.icon_06')[0].text
        number = i.select('li.icon_02')[0].text
        address = i.select('li.icon_01')[0].text
        name = i('strong')[0].text
        
        rest_list_data.append([menu, name, address, number])

In [ ]:
rest_list_df = pd.DataFrame(rest_list_data)
rest_list_df.columns = ['주메뉴', '이름', '주소', '연락처']

In [ ]:
rest_list_df

In [ ]:
rest_list_df.to_excel('Desktop/천안은 처음이라_먹거리.xlsx', index = False)

### 빵집 및 호두과자 크롤링

In [ ]:
res = requests.get('https://www.cheonan.go.kr/tour/sub03_12.do')
soup = BeautifulSoup(res.content, 'html.parser')
bakery_list = soup.select('div.con')

bakery_list_data = []

for bakery in bakery_list:
    name = bakery.select('strong.title2')[0].text
    inform = bakery.select('p.mb_10')[0].text.replace('\r', '').replace('\n', '').replace('\t', '')
    number = bakery.select('li')[1].text.split('/')[1].strip()
    address = bakery.select('li')[1].text.split('/')[0].split(':')[1].strip()
    
    bakery_list_data.append([name, inform, address, number])

In [ ]:
columns = ['이름', '소개', '주소', '연락처']
bakery_list_df = pd.DataFrame(bakery_list_data, columns = columns)

In [ ]:
bakery_list_df

In [ ]:
bakery_list_df.to_excel('Desktop/천안은 처음이라_빵집 및 호두과자.xlsx', index = False)

### 나머지 데이터 수집 결과 

In [ ]:
import pandas as pd
fastiver = pd.read_excel("Desktop/천안_축제.xlsx")
museum = pd.read_excel("Desktop/천안_박물관.xlsx")
san = pd.read_excel("Desktop/천안_산.xlsx")
cheonan12 = pd.read_excel("Desktop/천안_천안12경.xlsx")
nature = pd.read_excel("Desktop/천안_자연관광.xlsx")
camping = pd.read_excel("Desktop/천안_캠핑장.xlsx")

In [ ]:
fastiver

In [ ]:
museum

In [ ]:
san

In [ ]:
cheonan12

In [ ]:
nature

In [ ]:
camping

### 도로명주소를 통해 위도와 경도로 반환

In [ ]:
bakery = pd.read_excel("Desktop/천안_빵집 및 호두과자.xlsx")

In [ ]:
address = bakery['주소']

In [ ]:
for i in range(len(address)):
    a = address[i].split(' ')
    address[i] = " ".join(a[0:3])
print(address)

In [ ]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent = 'South Korea')
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    
    except:
        return[0, 0]

In [ ]:
latitude = []
longitude = []

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [ ]:
bakery['위도'] = latitude
bakery['경도'] = longitude

In [ ]:
bakery